## Домашнее задание по Автоматической обработке естественного языка №2

### Настя Шабаева БКЛ181

#### Сравнение нескольких морфологических анализаторов

### Часть 2. Английский язык

## Установка всех необходимых пакетов

In [ ]:
! pip install flair

In [ ]:
!pip install datascience

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
!python -m nltk.downloader averaged_perceptron_tagger

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Импорт всего необходимого

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tag import pos_tag
import spacy
nlp = spacy.load("en_core_web_sm")
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter

from sklearn.metrics import accuracy_score
import pandas as pd
pd.options.display.max_rows = 150
pd.set_option('display.expand_frame_repr', False)
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Сам текст

Я постаралась добавить в него несколько слов, которые в разных контекстах могут являться примерами разных частей речи (например, human - существительное и прилагательное, reserved - прилагательное и глагол и др). Стоит отметить, что ни один из анализаторов не отметил, например, human существительным в том контексте, где я считала его существительным. Если честно, в данном случае я не уверена, в чьей разметке проблема. 

In [ ]:
eng_text = '''In case of legal proceedings, these papers would only obscure the case. 
Some of the men were famous, others obscure. I hereby separate the whales from the fish. 
Elsewhere, private vehicles are rare and those that are used have separate tunnels reserved for them. 
He invited me to a fish restaurant. He was the opposite of his brother in almost every respect, being unusually shy and reserved. 
The train is going to go in the opposite direction. Tell me, are you human, or are you more than human? 
Human nature is much the same everywhere, is not it?
And they fear nothing, and they respect nothing, the young do not. Oddly, he felt no fear.'''
print(eng_text)

In case of legal proceedings, these papers would only obscure the case. 
Some of the men were famous, others obscure. I hereby separate the whales from the fish. 
Elsewhere, private vehicles are rare and those that are used have separate tunnels reserved for them. 
He invited me to a fish restaurant. He was the opposite of his brother in almost every respect, being unusually shy and reserved. 
The train is going to go in the opposite direction. Tell me, are you human, or are you more than human? 
Human nature is much the same everywhere, is not it?
And they fear nothing, and they respect nothing, the young do not. Oddly, he felt no fear.


## Словарь с размеченными токенами

Честно говоря, я не до конца уверена в своей разметке. Для некоторых слов мне было трудно определить часть речи. Например, тегг DET я ставила довольно интуитивно, не уверена, что получилось точно.

In [ ]:
eng_token_dict = {1 : {'in' : 'PR'},
                  2 : {'case' : 'NOUN'},
                  3 : {'of' : 'PR'},
                  4 : {'legal' : 'ADJ'},
                  5 : {'proceeings' : 'NOUN'},
                  6 : {'these' : 'DET'},
                  7 : {'papers' : 'NOUN'},
                  8 : {'would' : 'VERB'},
                  9 : {'only' : 'ADV'},
                  10 : {'obscure' : 'VERB'},
                  11 : {'the' : 'DET'},
                  12 : {'case' : 'NOUN'},
                  13 : {'some' : 'DET'},
                  14 : {'of' : 'PR'},
                  15 : {'the' : 'DET'},
                  16 : {'men' : 'NOUN'},
                  17 : {'were' : 'VERB'},
                  18 : {'famous' : 'ADJ'},
                  19 : {'others' : 'PRON'},
                  20 : {'obscure' : 'ADJ'},
                  21 : {'i' : 'PRON'},
                  22 : {'hereby' : 'ADV'},
                  23 : {'separate' : 'VERB'},
                  24 : {'the' : 'DET'},
                  25 : {'whales' : 'NOUN'},
                  26 : {'from' : 'PR'},
                  27 : {'the' : 'DET'},
                  28 : {'fish' : 'NOUN'},
                  29 : {'elsewhere' : 'ADV'},
                  30 : {'private' : 'ADJ'},
                  31 : {'vehicles' : 'NOUN'},
                  32 : {'are' : 'VERB'},
                  33 : {'rare' : 'ADJ'},
                  34 : {'and' : 'CCONJ'},
                  35 : {'those' : 'PRON'},
                  36 : {'that' : 'CCONJ'},
                  37 : {'are' : 'VERB'},
                  38 : {'used' : 'VERB'},
                  39 : {'have' : 'VERB'},
                  40 : {'separate' : 'ADJ'},
                  41 : {'tunnels' : 'NOUN'},
                  42 : {'reserved' : 'VERB'},
                  43 : {'for' : 'PR'},
                  44 : {'them' : 'PRON'},
                  45 : {'he' : 'PRON'},
                  46 : {'invited' : 'VERB'},
                  47 : {'me' : 'PRON'},
                  48 : {'to' : 'PR'},
                  49 : {'a' : 'DET'},
                  50 : {'fish' : 'ADJ'},
                  51 : {'restaurant' : 'NOUN'},
                  52 : {'he' : 'PRON'},
                  53 : {'was' : 'VERB'},
                  54 : {'the' : 'DET'},
                  55 : {'opposite' : 'NOUN'},
                  56 : {'of' : 'PR'},
                  57 : {'his' : 'PRON'},
                  58 : {'brother' : 'NOUN'},
                  59 : {'in' : 'PR'},
                  60 : {'almost' : 'ADV'},
                  61 : {'every' : 'DET'},
                  62 : {'respect' : 'NOUN'},
                  63 : {'being' : 'VERB'},
                  64 : {'unusually' : 'ADV'},
                  65 : {'shy' : 'ADJ'},
                  66 : {'and' : 'CCONJ'},
                  67 : {'reserved' : 'ADJ'},
                  68 : {'the' : 'DET'},
                  69 : {'train' : 'NOUN'},
                  70 : {'is' : 'VERB'},
                  71 : {'going' : 'VERB'},
                  72 : {'to' : 'PART'},
                  73 : {'go' : 'VERB'},
                  74 : {'in' : 'PR'},
                  75 : {'the' : 'DET'},
                  76 : {'opposite' : 'ADJ'},
                  77 : {'direction' : 'NOUN'},
                  78 : {'tell' : 'VERB'},
                  79 : {'me' : 'PRON'},
                  80 : {'are' : 'VERB'},
                  81 : {'you' : 'PRON'},
                  82 : {'human' : 'NOUN'},
                  83 : {'or' : 'CCONJ'},
                  84 : {'are' : 'VERB'},
                  85 : {'you' : 'PRON'},
                  86 : {'more' : 'ADV'},
                  87 : {'than' : 'CCONJ'},
                  88 : {'human' : 'NOUN'},
                  89 : {'human' : 'ADJ'},
                  90 : {'nature' : 'NOUN'},
                  91 : {'is' : 'VERB'},
                  92 : {'much' : 'ADV'},
                  93 : {'the' : 'DET'},
                  94 : {'same' : 'ADJ'},
                  95 : {'everywhere' : 'ADV'},
                  96 : {'is' : 'VERB'},
                  97 : {'not' : 'PART'},
                  98 : {'it' : 'PRON'},
                  99 : {'and' : 'CCONJ'},
                  100 : {'they' : 'PRON'},
                  101 : {'fear' : 'VERB'},
                  102 : {'nothing' : 'PRON'},
                  103 : {'and' : 'CCONJ'},
                  104 : {'they' : 'PRON'},
                  105 : {'respect' : 'VERB'},
                  106 : {'nothing' : 'PRON'},
                  107 : {'the' : 'DET'},
                  108 : {'young' : 'NOUN'},
                  109 : {'do' : 'VERB'},
                  110 : {'not' : 'PART'},
                  111 : {'oddly' : 'ADV'},
                  112 : {'he' : 'PRON'},
                  113 : {'felt' : 'VERB'},
                  114 : {'no' : 'DET'},
                  115 : {'fear' : 'NOUN'}}

### Вынимаем из словаря список только с частеречными теггами

In [ ]:
real_list = []

for i in eng_token_dict.values():
    for j in i.values():
        real_list.append(j)

### Работа со SpaCy

Обрабатываем текст с помощью SpaCy, потом проходим по всем частеречным теггам, меняем те названия, которые не совпали с моими. На основе полученного списка и "реального списка" считаем Accuracy

In [ ]:
doc = nlp(eng_text)

same_taggs = ['NOUN', 'VERB', 'ADV', 'ADJ', 'DET', 'PART', 'PRON', 'CCONJ']
spacy_list = []


for i, s in enumerate(doc.sents):
    for t in s:
        if t.pos_ != 'PUNCT':
            if t.pos_ in same_taggs:
                spacy_list.append(t.pos_)
            else:
                if t.pos_ == 'ADP':
                    spacy_list.append('PR')
                else:
                    if t.pos_ == 'AUX':
                        spacy_list.append('VERB')
                    else:
                        if t.pos_ == 'SCONJ':
                            spacy_list.append('CCONJ')

print('Accuracy: ', accuracy_score(real_list, spacy_list))

Accuracy:  0.9217391304347826


### Функция для переименовывания теггов, записанных в формате NLTK и flair.

In [ ]:
def change_tags(list_tags):
    list_append = []
    for i in list_tags:
        if (i[1] == 'IN') or (i[1] == 'TO'):
            list_append.append('PR')
        else:
            if (i[1] == 'NN') or (i[1] == 'NNS') or (i[1] == 'NNP') or (i[1] == 'NNPS'):
                list_append.append('NOUN')
            else:
                if (i[1] == 'JJ') or (i[1] == 'JJS') or (i[1] == 'JJR'):
                    list_append.append('ADJ')
                else:
                    if (i[1] == 'RB') or (i[1] == 'RBR') or (i[1] == 'RBS') or (i[1] == 'PDT') or (i[1] == 'WRB'):
                        list_append.append('ADV')
                    else:
                        if i[1] == 'DT':
                            list_append.append('DET')
                        else:
                            if (i[1] == 'MD') or (i[1] == 'VB') or (i[1] == 'VBD') or (i[1] == 'VBZ') or (i[1] == 'VBP') or (i[1] == 'VBG') or (i[1] == 'VBN'):
                                list_append.append('VERB')
                            else:
                                if (i[1] == 'PRP') or (i[1] == 'PRP$') or (i[1] == 'WP') or (i[1] == 'WP$'):
                                    list_append.append('PRON')
                                else:
                                    if (i[1] == 'WDT') or (i[1] == 'CC'):
                                        list_append.append('CCONJ')
                                    else:
                                        if (i[1] == 'RP'):
                                            list_append.append('PART')
    return list_append

### Обработка текста в NLTK

Токенизируем текст, обрабатываем в nltk.pos_tag, а дальше отправляем в функцию, меняющую имена теггов. На основе полученного списка и "реального списка" подсчитываем Accuracy

In [85]:
nltk_list = []
nltk_list = change_tags(nltk.pos_tag(nltk.word_tokenize(eng_text)))


print('Accuracy: ', accuracy_score(real_list, nltk_list))

Accuracy:  0.8608695652173913


### Обработка с помощью flair

С помощью flair текст делим на предложения, обрабатываем предложения, с помощью регулярных выражений вынимаем из них тегги.
Дальше меняем формат списка с теггами (для функции, которая переименовывает нужно, чтобы поступал список списков, в котором тегг на 2м месте. Отправляем полученный список в функцию, которая переименовывает тегги. И на основе полученного списка и "реального" считаем Accuracy

In [86]:
pos = SequenceTagger.load('pos-fast')
splitter = SegtokSentenceSplitter()

sentences = splitter.split(eng_text)
flair_list = []
tags = []
tags_new = []
help_l = []
pos.predict(sentences)

for sentence in sentences:
    tags.extend(re.findall(r'<(.+?)>', sentence.to_tagged_string()))

for i in range(len(tags)):
    help_l = []
    help_l.append(i)
    help_l.append(tags[i])
    tags_new.append(help_l)

flair_list = change_tags(tags_new)
print('Accuracy: ', accuracy_score(real_list, flair_list))

2020-10-16 18:20:01,711 loading file /root/.flair/models/en-pos-ontonotes-fast-v0.5.pt
Accuracy:  0.8956521739130435


### Составление таблицы

Создаем датафрейм с отдельными колонками для слова, тегга по моей разметке, тегга по разметке SpaCy, тегга по разметке NLTK и тегга по разметке flair.

В конце выводим Accuracy для каждого анализатора.

In [87]:
data = {'Word' : [], 'my POS' : [], 'spacy POS' : [], 'nltk POS' : [], 'flair POS' : []}

for i in range(len(real_list)):
    data['Word'].append(list(eng_token_dict[i+1].keys())[0])
    data['my POS'].append(real_list[i])
    data['spacy POS'].append(spacy_list[i])
    data['nltk POS'].append(nltk_list[i])
    data['flair POS'].append(flair_list[i])

df = pd.DataFrame(data)

print(df)
print('Accuracy spacy: ', accuracy_score(real_list, spacy_list))
print('Accuracy nltk: ', accuracy_score(real_list, nltk_list))
print('Accuracy flair: ', accuracy_score(real_list, flair_list))

           Word my POS spacy POS nltk POS flair POS
0            in     PR        PR       PR        PR
1          case   NOUN      NOUN     NOUN      NOUN
2            of     PR        PR       PR        PR
3         legal    ADJ       ADJ      ADJ       ADJ
4    proceeings   NOUN      NOUN     NOUN      NOUN
5         these    DET       DET      DET       DET
6        papers   NOUN      NOUN     NOUN      NOUN
7         would   VERB      VERB     VERB      VERB
8          only    ADV       ADV      ADV       ADV
9       obscure   VERB      VERB     VERB      VERB
10          the    DET       DET      DET       DET
11         case   NOUN      NOUN     NOUN      NOUN
12         some    DET       DET      DET       DET
13           of     PR        PR       PR        PR
14          the    DET       DET      DET       DET
15          men   NOUN      NOUN     NOUN      NOUN
16         were   VERB      VERB     VERB      VERB
17       famous    ADJ       ADJ      ADJ       ADJ
18       oth